In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import json
import nltk
import gensim
import numpy as np
import pandas as pd
from gensim import corpora, models, similarities
import pickle
import csv
import re
import tensorflow as tf

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
os.chdir("/content/drive/My Drive/Deep Learning")

In [0]:
file=open('/content/drive/My Drive/Deep Learning/conversation.json')
data = json.load(file)

In [0]:
x = list()
y = list()

In [0]:
conversations = data['conversations']

In [0]:
for conversation in conversations:
  for position in range(len(conversation)):
    if position < (len(conversation)-1):
      x.append(conversation[position])
      y.append(conversation[position+1])

In [0]:
df = pd.DataFrame(list(zip(x, y)), columns = ['Question' , 'Answer'])

In [0]:
df['Answer'] = "startseq " + df['Answer'] + " endseq"

In [0]:
#Cleaning the Train Dataset
df['Question'] = df['Question'].apply(lambda x: x.lower())
df['Answer'] = df['Answer'].apply(lambda x: x.lower())


In [0]:
df['Question']  = df['Question'].str.replace("won't","will not")
df['Question']  = df['Question'].str.replace("ain't","am not")
df['Question']  = df['Question'].str.replace("'s"," is")
df['Question']  = df['Question'].str.replace("'m"," am")
df['Question']  = df['Question'].str.replace("'re'"," are")
df['Question']  = df['Question'].str.replace("can't","can not")
df['Question']  = df['Question'].str.replace("'ll"," will")
df['Question']  = df['Question'].str.replace("n't"," not")

df['Answer']  = df['Answer'].str.replace("won't","will not")
df['Answer']  = df['Answer'].str.replace("ain't","am not")
df['Answer']  = df['Answer'].str.replace("'s"," is")
df['Answer']  = df['Answer'].str.replace("'m"," am")
df['Answer']  = df['Answer'].str.replace("'re'"," are")
df['Answer']  = df['Answer'].str.replace("can't","can not")
df['Answer']  = df['Answer'].str.replace("'ll"," will")
df['Answer']  = df['Answer'].str.replace("n't"," not")

In [0]:
df['Question'] = df['Question'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
df['Question'] = df['Question'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['Answer'] = df['Answer'].apply(lambda x: re.sub('[^a-zA-z\s]','',x)) #removing all charecters except alphabets
df['Answer'] = df['Answer'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces

In [0]:
df1 = df.copy()

In [0]:

"""The num_words in Tokenizer, the maximum number of words to keep, based on word frequency. 
If num_words = 10000 Only the most common 9999 words will be kept. all he extra words will be removed."""
q_tokenizer = Tokenizer() #setting up tokenizer
a_tokenizer = Tokenizer() 
q_tokenizer.fit_on_texts(df['Question']) #fitting tokenizer on dataframe 
a_tokenizer.fit_on_texts(df['Answer'])
df['Question'] = q_tokenizer.texts_to_sequences(df['Question']) # removing least repeated words and converting them into sequence of numbers.
df['Answer'] = a_tokenizer.texts_to_sequences(df['Answer'])

In [0]:
encoder_input = []
decoder_input = []
decoder_output = []
for i in range(len(df['Question'])):
  temp_list = []
  for j in range(len(df['Answer'][i])):
    if j < (len(df['Answer'][i])-1):
      temp_list.append(df['Answer'][i][j])
      encoder_input.append(df['Question'][i])
      decoder_input.append((temp_list).copy())
      decoder_output.append(df['Answer'][i][j+1])

In [17]:
df1.head()

,Question,Answer
0,good morning how are you,startseq i am doing well how about you endseq
1,i am doing well how about you,startseq i am also good endseq
2,i am also good,startseq that is good to hear endseq
3,that is good to hear,startseq yes it is endseq
4,hello,startseq hi endseq


In [18]:
#Encoder Sequence Length Maximum
encoder_sequence_length = max([len(x) for x in encoder_input])
encoder_sequence_length

28

In [19]:
#Decoder Sequence Length Maximum
decoder_sequence_length = max([len(x) for x in decoder_input])
decoder_sequence_length

29

In [0]:
encoder_input = pad_sequences(encoder_input,maxlen = encoder_sequence_length, padding='pre')
decoder_input = pad_sequences(decoder_input,maxlen = decoder_sequence_length, padding='post')

In [0]:
num_classes = len(a_tokenizer.word_index)+1

In [0]:
num_tokens = len(q_tokenizer.word_index)+1

In [23]:
from keras.utils import to_categorical
decoder_output = to_categorical(decoder_output,num_classes=num_classes)

Using TensorFlow backend.


In [24]:
encoder_input.shape,decoder_input.shape,decoder_output.shape

((609, 28), (609, 29), (609, 221))

In [25]:
#Model Creation - Functional Type - Encoder
# Define an input sequence and process it.
encoder_Inputs = tf.keras.layers.Input(shape=(None,))
encoder_embedding = tf.keras.layers.Embedding( num_tokens, 200 , mask_zero=True) (encoder_Inputs)
encoder = tf.keras.layers.LSTM(256,return_state=True)
encoder_outputs, hidden_state, cell_state = encoder(encoder_embedding)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [hidden_state, cell_state]

W0703 07:35:05.232491 140182924056448 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 07:35:05.252631 140182924056448 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 07:35:05.428966 140182924056448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:3794: add_dispatch_support.<locals>.wrapper (from tensorflow.pytho

In [0]:
#Model Creation - Functional Type - Decoder
decoder_Inputs = tf.keras.layers.Input(shape=(None,))
decoder_embedding = tf.keras.layers.Embedding( num_classes, 200 , mask_zero=True) (decoder_Inputs)
decoder = tf.keras.layers.LSTM(256)
decoder_outputs = decoder(decoder_embedding,initial_state = encoder_states)
decoder_final_output = tf.keras.layers.Dense(num_classes, activation= 'softmax')(decoder_outputs)

In [0]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
chatmodel = tf.keras.Model([encoder_Inputs, decoder_Inputs], decoder_final_output)
chatmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
chatmodel.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    41600       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    44200       input_2[0][0]                    
______________________________________________________________________________________________

In [29]:
chatmodel.fit([encoder_input, decoder_input], decoder_output, batch_size=487, epochs=100)

Epoch 1/100
609/609 [==============================] - 2s 4ms/sample - loss: 5.3975
Epoch 2/100
609/609 [==============================] - 0s 336us/sample - loss: 5.3822
Epoch 3/100
609/609 [==============================] - 0s 319us/sample - loss: 5.3634
Epoch 4/100
609/609 [==============================] - 0s 332us/sample - loss: 5.3320
Epoch 5/100
609/609 [==============================] - 0s 312us/sample - loss: 5.2697
Epoch 6/100
609/609 [==============================] - 0s 320us/sample - loss: 5.1305
Epoch 7/100
609/609 [==============================] - 0s 317us/sample - loss: 4.9694
Epoch 8/100
609/609 [==============================] - 0s 338us/sample - loss: 4.7935
Epoch 9/100
609/609 [==============================] - 0s 324us/sample - loss: 4.6396
Epoch 10/100
609/609 [==============================] - 0s 313us/sample - loss: 4.5569
Epoch 11/100
609/609 [==============================] - 0s 310us/sample - loss: 4.5141
Epoch 12/100
609/609 [==============================] 

In [0]:
index_word = dict([(index,word) for word, index in a_tokenizer.word_index.items()])

In [31]:
index_word[np.argmax(chatmodel.predict([encoder_input[0:1], decoder_input[0:1]])[0])]

'i'

In [32]:
chatmodel.predict([encoder_input[0:1], decoder_input[1:2]])

array([[7.68661891e-07, 1.53209066e-06, 1.26038913e-05, 3.04700916e-05,
        9.00607090e-03, 5.03747333e-06, 3.54648109e-05, 1.08387438e-04,
        9.17531142e-04, 2.59445951e-04, 9.08671856e-01, 3.80284519e-07,
        4.29922147e-05, 3.37326037e-06, 1.30568182e-07, 6.28507347e-04,
        1.32555622e-04, 3.75810341e-05, 3.06303045e-05, 1.40959062e-04,
        5.17665067e-05, 2.61155537e-06, 1.54104564e-04, 8.03494549e-05,
        3.41283367e-03, 3.82777507e-04, 2.13984633e-03, 6.98690928e-05,
        4.25333055e-05, 1.26926752e-04, 4.72695456e-06, 2.89137489e-07,
        4.57916111e-02, 2.06675706e-03, 2.86144336e-06, 7.48400589e-06,
        2.99445470e-04, 6.95013223e-05, 7.50756044e-06, 6.28025082e-05,
        1.62155322e-06, 9.65221943e-07, 5.26669737e-07, 1.12517818e-03,
        4.11774468e-04, 4.30375667e-06, 2.48695794e-03, 5.59347609e-06,
        2.52327212e-04, 2.96200765e-03, 2.91372459e-07, 9.57590510e-06,
        1.37158167e-05, 5.19590321e-06, 1.70604073e-06, 1.118385

In [0]:
predicted = chatmodel.predict([encoder_input[0:1], decoder_input[0:1]])[0]

In [34]:
temp_list = list()
for i in index_word:
  temp_list.append((index_word[i],predicted[i]))
temp_list

[('startseq', 4.2871773e-07),
 ('endseq', 3.0892413e-06),
 ('is', 4.787392e-05),
 ('i', 0.89737684),
 ('you', 7.978773e-06),
 ('it', 0.00025416716),
 ('the', 5.963321e-05),
 ('a', 0.0029208963),
 ('what', 0.0053994786),
 ('am', 0.0022286035),
 ('to', 4.417247e-07),
 ('of', 1.3232672e-05),
 ('better', 5.8246746e-07),
 ('than', 2.4898847e-07),
 ('that', 0.0016448073),
 ('do', 0.00031943247),
 ('good', 3.408443e-05),
 ('are', 0.0004114621),
 ('not', 7.454944e-06),
 ('how', 0.00040587567),
 ('your', 6.8487793e-07),
 ('but', 2.5210538e-05),
 ('like', 2.2653412e-05),
 ('doing', 0.00010663373),
 ('yes', 0.052046634),
 ('in', 0.00032499855),
 ('complex', 0.00036114446),
 ('and', 3.4929121e-06),
 ('well', 7.2350394e-06),
 ('about', 8.317091e-06),
 ('hear', 3.6476658e-07),
 ('can', 0.0023233446),
 ('who', 0.00865391),
 ('tell', 1.46612e-06),
 ('know', 2.4988706e-06),
 ('complicated', 0.00012641241),
 ('python', 1.6902006e-05),
 ('although', 0.00016257036),
 ('never', 4.5591023e-06),
 ('be', 3.29

In [35]:
np.argmax(predicted)

4

In [36]:
len(a_tokenizer.word_index)+1

221

In [37]:
decoder_sequence = a_tokenizer.texts_to_sequences(["startseq"])[0]
pad_sequences([decoder_sequence],maxlen=29,padding="pre")

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1]], dtype=int32)

In [0]:
import heapq
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import json
import nltk
import gensim
import numpy as np
import pandas as pd
from gensim import corpora, models, similarities
import pickle
import csv
import re
import tensorflow as tf
class Beam(object):
    """
    For comparison of prefixes, the tuple (prefix_prob, complete_sentence) is uesd.
    This is so that if two prefixes have eequal probabilities then a complete sentence is preffered
    over and incomplete one since (0.5, False) < (0.5, True)
    """
    
    def __init__(self, beam_width):
        self.heap = []
        self.beam_width = beam_width
    
    def add(self, prob, complete, prefix):
        heapq.heappush(self.heap, (prob, complete, prefix))

        # if we exeed the beam size, remove the "worst" item
        if len(self.heap) > self.beam_width:
            heapq.heappop(self.heap)
    
    def __iter__(self):
        return iter(self.heap)
    
    def __repr__(self):
        return str(self.heap)
      
def beamsearch(probabilities_function, encoder_input, start_word, encoder_tokenizer, question_length, decoder_tokenizer, index_to_word_dict, model, beam_width=3, clip_len=-1, Question_Padding= "pre"):
    # Probablities Function- Function which output the probablity of each word predicted.
    prev_beam = Beam(beam_width)
    prev_beam.add(1.0, False, [ start_word ])
    encoder_sequence = encoder_tokenizer.texts_to_sequences([encoder_input])[0]
    encoder_sequence = pad_sequences([encoder_sequence],maxlen=question_length,padding=Question_Padding)
    while True:
        curr_beam = Beam(beam_width)
         
        #Add complete sentences that do not yet have the best probability to the current beam, the rest prepare to add more words to them.
        for (prefix_prob, complete, prefix) in prev_beam:
            if complete == True:
                curr_beam.add(prefix_prob, True, prefix)
            else:
                #Get probability of each possible next word for the incomplete prefix.
                for (next_prob, next_word) in probabilities_function(encoder_sequence, prefix, decoder_tokenizer, index_to_word_dict, model):
                    if next_word == 'endseq': #if next word is the end token then mark prefix as complete and leave out the end token
                        curr_beam.add(prefix_prob*next_prob, True, prefix)
                    else: #if next word is a non-end token then mark prefix as incomplete
                        curr_beam.add(prefix_prob*next_prob, False, prefix+[next_word])
         
        (best_prob, best_complete, best_prefix) = max(curr_beam)
        if best_complete == True or len(best_prefix)-1 == clip_len: #if most probable prefix is a complete sentence or has a length that exceeds the clip length (ignoring the start token) then return it
            return (best_prefix[1:], best_prob) #return best sentence without the start token and together with its probability
             
        prev_beam = curr_beam

In [0]:
# beam search
def beam_search_decoder_single_word_probabilities_function(encoder_sequence,decoder_word, decoder_tokenizer, index_to_word_dict, model):
  decoder_sequence = decoder_tokenizer.texts_to_sequences([decoder_word])[0]
  decoder_sequence = pad_sequences([decoder_sequence],maxlen=29,padding="pre") 
	
  predicted = model.predict([encoder_sequence, decoder_sequence])[0]
  probablity_word_tuple_list = list()
  for i in index_word:
    probablity_word_tuple_list.append((predicted[i],index_word[i]))
  return probablity_word_tuple_list

In [45]:
beamsearch(beam_search_decoder_single_word_probabilities_function, "what is your favorite book", 'startseq', q_tokenizer, 30, a_tokenizer, index_word, chatmodel)

(['i', 'can', 'not', 'read'], 0.14852316276359784)

In [0]:
# beamsearch(probabilities_function, encoder_input, start_word, encoder_tokenizer, question_length, decoder_tokenizer, index_to_word_dict, model, beam_width=3, clip_len=-1, Question_Padding= "pre")

In [0]:
index_list = predicted.argsort()[-3:][::-1]
for i in index_list:
  print(predicted[i],index_word[i])

0.99933535 i
0.00018951378 yes
0.00018746688 how


In [0]:
[encoder_input[0:1], decoder_input[0:6]]

[array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  1,  4, 10, 24, 29, 20, 30,  5,  2]],
       dtype=int32),
 array([[ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  4, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  4, 10, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  4, 10, 24, 29,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1,  4, 10, 24, 29, 20,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)]